In [ ]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

In [ ]:
def make_table(width, height, value=[0.00]):
    return [value * width for _ in range(height)] 

In [ ]:
def make_uniform_dist(num):
    return [1/num] * num

In [ ]:
import queue as Q

class MaxPriorityQueue(Q.PriorityQueue):
    def inverse_first(self, item):
        return (-1*item[0], item[1])
        
    def get(self, block=True, timeout=None):
        item = super(MaxPriorityQueue, self).get()
        return self.inverse_first(item)
    
    def put(self, item, block=True, timeout=None):
        item = self.inverse_first(item)
        super(MaxPriorityQueue, self).put(item) 
        
    def max_idxs(self):
        idxs, top = [], None
        while not self.empty():
            item = self.get()
            if top == None:
                top = item
            if top[0] > item[0]:
                return idxs
            idxs.append(item[1])
        return idxs

In [ ]:
mq = MaxPriorityQueue()
mq.put((3, 1))
mq.put((9, 3))
mq.put((6, 0))
mq.put((0, 2))
mq.max_idxs()

In [ ]:
from environment_value import GraphicDisplay, Env

In [ ]:
class ValueIteration:
    def __init__(self, env):
        self.env = env
        self.discount_factor = 0.9
        self.value_table = make_table(env.width, env.height)
    
    def is_final_state(self, state):
        return state == [2, 2]
        
    def get_value(self, state):
        return round(self.value_table[state[0]][state[1]], 2)
    
    def get_value_to_action(self, state):
        value_to_action = MaxPriorityQueue()
        for action in env.possible_actions:
            next_state, next_value, reward = self.interact_env(state, action)
            value = reward + self.discount_factor*next_value
            value_to_action.put((value, action))
        return value_to_action
        
    def interact_env(self, state, action):
        next_state = self.env.state_after_action(state, action)
        next_value = self.get_value(next_state)
        reward = self.env.get_reward(state, action)
        return next_state, next_value, reward
    
    def get_action(self, state):
        if self.is_final_state(state):
            return []
        
        value_to_action = self.get_value_to_action(state)
        return value_to_action.max_idxs()
    
    def value_iteration(self):
        env = self.env
        next_value_table = make_table(env.width, env.height)
        
        def update_next_value(state, value):
            next_value_table[state[0]][state[1]] = value
        
        for state in env.get_all_states():
            if self.is_final_state(state):
                update_next_value(state, 0)
                continue

            value_to_action = self.get_value_to_action(state)
            optimal_value = value_to_action.get()[0]
            update_next_value(state, round(optimal_value, 2))
        
        self.value_table = next_value_table

In [ ]:
env = Env()
value_iteration = ValueIteration(env)
grid_world = GraphicDisplay(value_iteration)
grid_world.mainloop()